In [1]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Basic Recording Example
## Recording Terminology
- **Case**: A Case stores a snapshot of all the variable values, metadata, and options of a model, or a sub-set of a model, at a particular point in time
- **Case Recorder**: An OpenMDAO module used to store a snapshot of a model before, during, or after execution in an SQL file.
- **Sources**: The OpenMDAO object responsible for recording the case. Can be `Problem`, `Driver` or a specific `System` or `Solver` identified by pathname.

## Basic Recording Example

Below is a basic example of how to create a recorder, attach it to a Problem, save the information,
and retrieve the data from the recorder. `list_outputs` is a quick way to show all of your outputs
and their values at the time the case was recorded, and should you need to isolate a single value OpenMDAO provides two ways to
retrieve them. To view all the design variables, constraints, and
objectives, you can use their methods like the example below.


In [2]:
import os
if os.path.exists('cases.sql'):
    os.remove('cases.sql')

In [3]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src8", get_code("openmdao.test_suite.components.sellar_feature.SellarMDAWithUnits"), display=False)

:::{Admonition} `SellarMDAWithUnits` class definition 
:class: dropdown

{glue:}`code_src8`
:::

In [7]:
from openmdao.test_suite.components.sellar_feature import SellarMDAWithUnits
import numpy as np
import openmdao.api as om

# build the model
prob = om.Problem(model=SellarMDAWithUnits())

model = prob.model
model.add_design_var('z', lower=np.array([-10.0, 0.0]), upper=np.array([10.0, 10.0]))
model.add_design_var('x', lower=0.0, upper=10.0)
model.add_objective('obj')
model.add_constraint('con1', upper=0.0)
model.add_constraint('con2', upper=0.0)

# setup the optimization
driver = prob.driver = om.ScipyOptimizeDriver(optimizer='SLSQP', tol=1e-9, disp=False)

# Create a recorder variable
recorder = om.SqliteRecorder('cases.sql')
# Attach a recorder to the problem
prob.add_recorder(recorder)

prob.setup()
prob.set_solver_print(2)
prob.run_driver()
prob.record("after_run_driver")

# Instantiate your CaseReader
cr = om.CaseReader(prob.get_outputs_dir() / "cases.sql")
# Isolate "problem" as your source
driver_cases = cr.list_cases('problem', out_stream=None)
# Get the first case from the recorder
case = cr.get_case('after_run_driver')

# These options will give outputs as the model sees them
# Gets value but will not convert units
const = case['con1']
print(const)

# get_val can convert your result's units if desired
const_K = case.get_val("con1", units='K')

print(const_K)


=====
cycle
=====
NL: NLBGS 1 ; 29.0742299 1
NL: NLBGS 2 ; 2.26505979 0.0779060975
NL: NLBGS 3 ; 0.0438762115 0.00150911001
NL: NLBGS 4 ; 0.000867730413 2.98453446e-05
NL: NLBGS 5 ; 1.715381e-05 5.90000492e-07
NL: NLBGS 6 ; 3.39109478e-07 1.16635756e-08
NL: NLBGS 7 ; 6.70377103e-09 2.30574328e-10
NL: NLBGS 8 ; 1.32523072e-10 4.55809395e-12
NL: NLBGS Converged

=====
cycle
=====
NL: NLBGS 1 ; 2.62038164e-12 1
NL: NLBGS Converged

=====
cycle
=====
NL: NLBGS 1 ; 19.1706591 1
NL: NLBGS 2 ; 1.13589624 0.0592518093
NL: NLBGS 3 ; 0.0406301437 0.00211939211
NL: NLBGS 4 ; 0.00140619452 7.33513916e-05
NL: NLBGS 5 ; 4.87225696e-05 2.54151771e-06
NL: NLBGS 6 ; 1.68809957e-06 8.80564178e-08
NL: NLBGS 7 ; 5.84879656e-08 3.05091053e-09
NL: NLBGS 8 ; 2.0264448e-09 1.05705536e-10
NL: NLBGS 9 ; 7.02104123e-11 3.66238907e-12
NL: NLBGS Converged

=====
cycle
=====
NL: NLBGS 1 ; 5.26036063 1
NL: NLBGS 2 ; 0.514550635 0.0978166083
NL: NLBGS 3 ; 0.0258623712 0.00491646354
NL: NLBGS 4 ; 0.00126421196 0.0002

/Users/rfalck/Codes/OpenMDAO.git/openmdao/recorders/sqlite_recorder.py:226: UserWarning:The existing case recorder file, problem3_out/cases.sql, is being overwritten.


[-1.66901604e-10]
[273.15]


In [9]:
from openmdao.utils.assert_utils import assert_near_equal
assert_near_equal(const, -1.68550507e-10, 1e-8, tol_type='abs')
assert_near_equal(const_K, 273.15, 1e-3, tol_type='rel')

6.109913153041638e-13

In [10]:
# list_outputs will list your model's outputs and return a list of them too
print(case.list_outputs())

5 Explicit Output(s) in 'model'

varname   val                prom_name
--------  -----------------  ---------
cycle
  d1
    y1    [3.16]             y1       
  d2
    y2    [3.75527777]       y2       
obj_cmp
  obj     [3.18339395]       obj      
con_cmp1
  con1    [-1.66901604e-10]  con1     
con_cmp2
  con2    [-20.24472223]     con2     


0 Implicit Output(s) in 'model'


[('con_cmp1.con1', {'val': array([-1.66901604e-10]), 'io': 'output', 'prom_name': 'con1'}), ('con_cmp2.con2', {'val': array([-20.24472223]), 'io': 'output', 'prom_name': 'con2'}), ('cycle.d1.y1', {'val': array([3.16]), 'io': 'output', 'prom_name': 'y1'}), ('cycle.d2.y2', {'val': array([3.75527777]), 'io': 'output', 'prom_name': 'y2'}), ('obj_cmp.obj', {'val': array([3.18339395]), 'io': 'output', 'prom_name': 'obj'})]


In [11]:
# This code below will find all the objectives, design variables, and constraints that the
# problem source contains
objectives = case.get_objectives()
print(objectives['obj'])

design_vars = case.get_design_vars()
print(design_vars['x'])

constraints = case.get_constraints()
print(constraints['con1'])

[3.18339395]
[0.]
[-1.66901604e-10]


In [13]:
assert_near_equal(objectives['obj'], 3.18339395, 1e-4)
assert_near_equal(design_vars['x'], 0., 1e-4)
assert_near_equal(constraints['con1'], -1.68550507e-10, 1e-8, tol_type='abs')

1.6489033512608081e-12